## Project: AI-BEMS 알고리즘 도출
* 2024.10.11. SQI에서 2023.1. ~ 2024.10. 까지 BEMS 데이터 업데이트

In [1]:
# 라이브러리를 수정할 때마다 autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import pickle

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/buildingID_devID_map.csv"
buildingID_devID_df = pd.read_csv(file_path)
print(buildingID_devID_df.head(3))

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/devID_tagCD_map.csv"
devID_tagCD_df = pd.read_csv(file_path)
print(devID_tagCD_df.head(3))

  buildingID  devID                    devName devType
0      B0001   1001     인천해양과학고등학교 인버터1_2.53kW     인버터
1      B0001   1002     인천해양과학고등학교 인버터2_2.53kW     인버터
2      B0001   1003  인천해양과학고등학교 신축인버터1_49.95kW     인버터
       devID_range devID_description  tagCD         tagName
0  devID 1001~1999   inverter 태양광인버터  10002           PV 전압
1  devID 1001~1999   inverter 태양광인버터  10003  계통 R상(RS선간) 전압
2  devID 1001~1999   inverter 태양광인버터  10004  계통 S상(ST선간) 전압


In [2]:
import numpy as np
import pandas as pd
import Utility as Util

#########
buildingID, buildingName = 'B0008', '용학초등학교'
file_path = '/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/data_colec_h_202410141057_B0008.csv'
#########
df_buildID = pd.read_csv(file_path)
df_buildID['colec_dt'] = pd.to_datetime(df_buildID['colec_dt']).dt.floor('min')  # 분 이하는 제거
print(f"Start timestamp: first: {df_buildID['colec_dt'].iloc[0]}, min: {df_buildID['colec_dt'].min()}")
print(f"End timestamp: last: {df_buildID['colec_dt'].iloc[-1]}, max: {df_buildID['colec_dt'].max()}")

dev_id_list = df_buildID['dev_id'].unique()
print(f"{len(dev_id_list)=}\n{dev_id_list=}")
Inverter = np.sum((dev_id_list >= 1000) & (dev_id_list < 2000))
ElecMeter = np.sum((dev_id_list >= 2000) & (dev_id_list < 3000))
GasMeter = np.sum((dev_id_list >= 3000) & (dev_id_list < 4000))
HeatMeter = np.sum((dev_id_list >= 4000) & (dev_id_list < 5000))
EnvSensor = np.sum((dev_id_list >= 5000) & (dev_id_list < 6000))
AirCon = np.sum((dev_id_list >= 6000) & (dev_id_list < 7000))
print(f'{Inverter=}, {ElecMeter=}, {GasMeter=}, {HeatMeter=}, {EnvSensor=}, {AirCon=}')

df_buildID.head(3)

Start timestamp: first: 2023-01-01 00:00:00, min: 2023-01-01 00:00:00
End timestamp: last: 2024-10-14 10:30:00, max: 2024-10-14 10:30:00
len(dev_id_list)=133
dev_id_list=array([2001, 2002, 2003, 2004, 3001, 3002, 3003, 3004, 3005, 4001, 4002,
       5001, 5002, 5003, 1001, 1002, 6001, 6002, 6003, 6004, 6005, 6006,
       6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015, 6016, 6017,
       6018, 6019, 6020, 6021, 6022, 6023, 6024, 6025, 6026, 6027, 6028,
       6029, 6030, 6031, 6032, 6033, 6034, 6035, 6036, 6037, 6038, 6039,
       6040, 6041, 6042, 6043, 6044, 6045, 6046, 6047, 6048, 6049, 6050,
       6051, 6052, 6053, 6054, 6055, 6056, 6057, 6058, 6059, 6060, 6061,
       6062, 6063, 6064, 6065, 6066, 6067, 6068, 6069, 6070, 6071, 6072,
       6073, 6074, 6075, 6076, 6077, 6078, 6079, 6080, 6081, 6082, 6083,
       6084, 6085, 6086, 6087, 6088, 6089, 6090, 6091, 6092, 6093, 6094,
       6095, 6096, 6097, 6098, 6099, 6100, 6101, 6102, 6103, 6104, 6105,
       6106, 6107, 6108, 61

,dev_id,tag_cd,colec_dt,colec_val,reg_dt,bldg_id
0,2001,202202,2023-01-01,1.00,2023-01-01 00:13:15.700,B0008
1,2001,30001,2023-01-01,49.00,2023-01-01 00:13:15.700,B0008
2,2001,40004,2023-01-01,413502.81,2023-01-01 00:13:15.700,B0008


### 기간 설정

In [3]:
# 학사일정.... 용학초등학교
"""
2023년
03-02(목) ~ 07-21(금) : 입학식~여름방학식, semester1
07-22(토) ~ 08-21(월) : 여름방학시작~종료, summerVacation
08-22(화) ~ 12-29(금) : 여름방학개학식~겨울방학식, semester2
12-30(토) ~ 01-23(화) : 겨울방학시작~종료, winterVacation
01-24(수) ~ 01-31(수) : 겨울방학개학식~졸업식, winterSession 
02-01(목) ~ 03-03(일) : 졸업방학시작~종료, graduationVacation

03-04(월) ~ 07-24(수) : 입학식~여름방학식, semester1
"""

start_date, end_date = pd.to_datetime('2023-11-15 00:00:00'), pd.to_datetime('2023-12-16 00:00:00')

df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

### 학교 MAIN

In [6]:
#B0008,2001,학교 MAIN,전기계측기

devID, device_name = 2001, '학교MAIN'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량_차', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}  # 전체누적사용량_차는 패턴이 이상함
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = Util.calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'
#tags[40007] = calc_tagCD_diff(tags[40007])  # '역송전력량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)


tagCD list:  [202202  30001  40004  40007  50001]


### 인버터

In [13]:
#B0008,1001,인천용학초등학교 인버터1_28.8kW,PVES-100
#B0008,1002,인천용학초등학교 인버터2_102kW,PVES-030

devID, device_name = 1002, '인버터'
Util.print_tagCD(df, devID)
tag_dict = {10002: 'PV전압', 20002: 'PV전류', 30001: '현재출력', 30002: 'PV출력', 60001: '주파수', 70006: '인버터온도', 202202: 'validData'}
tag_dict = {30001: '현재출력'}

tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

tagCD list:  [ 202202   10002   10003   10004   10005   20002   20003   20004   20005
   30001   30002   40001   60001     703 9003005 9003007 9003008 9003014
 9003020 9003023 9003030 9003034 9003038 9003040 9003041 9003042 9003045
 9003047 9003048 9003049 9003081 9003106]


#### EHP

In [8]:
#B0008,2002,EHP-1,전기계측기
#B0008,2003,EHP-2,전기계측기
#B0008,2004,EHP-4,전기계측기

devID, device_name = 2002, 'EHP-2'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

tagCD list:  [202202  30001  40004  50001]


#### 전기 계측기

In [ ]:
# 보류

devID, device_name = 2003, '교사동EL'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 전기온수기

In [14]:
#보류

devID, device_name = 2081, '전기온수기'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

   buildingID  devID          devName devType
13      B0001   2010   교사 서측 1F 전기온수기   전기계측기
27      B0001   2024  교사 중앙 1F 전기온수기1   전기계측기
28      B0001   2025  교사 중앙 1F 전기온수기2   전기계측기
29      B0001   2026  교사 중앙 1F 전기온수기3   전기계측기
33      B0001   2030  교사 중앙 2F 전기온수기1   전기계측기
34      B0001   2031  교사 중앙 2F 전기온수기2   전기계측기
35      B0001   2032  교사 중앙 2F 전기온수기3   전기계측기
39      B0001   2036  교사 중앙 3F 전기온수기1   전기계측기
40      B0001   2037  교사 중앙 3F 전기온수기2   전기계측기
41      B0001   2038  교사 중앙 3F 전기온수기3   전기계측기
45      B0001   2042  교사 중앙 4F 전기온수기1   전기계측기
46      B0001   2043  교사 중앙 4F 전기온수기2   전기계측기
48      B0001   2045     체육관 급탕 전기온수기   전기계측기
66      B0001   2063    실습1동 1F 전기온수기   전기계측기
70      B0001   2067    실습1동 2F 전기온수기   전기계측기
74      B0001   2071    실습1동 3F 전기온수기   전기계측기
78      B0001   2075   실습2동 1F 전기온수기1   전기계측기
79      B0001   2076   실습2동 1F 전기온수기2   전기계측기
80      B0001   2077   실습2동 1F 전기온수기3   전기계측기
84      B0001   2081    실습2동 2F 전기온수기   전기계측기
tagCD list:  [202202  30001  40004

#### 전등

In [ ]:
# 보류

devID, device_name = 2062, '전등'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 환경 센서

In [ ]:
#B0008,5001,외기,환경센서
#B0008,5002,1층특수학급내부 ,환경센서
#B0008,5003,2층복도,환경센서

devID, device_name = 5003, '온도/습도/CO2/초미세먼지/미세먼지/조도'
Util.print_tagCD(df, devID)
tag_dict = {80002: '온도', 80003: '습도', 80004: 'CO2', 80005: '미세먼지', 80006: '초미세먼지', 80007: '조도', 202202: 'validData'}
tag_dict = {80002: '온도'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 배기휀

In [ ]:
# 보류

devID, device_name = 2051, '체육관 B1F 탁구장 배기휀   전기계측기'  # 작동없음
devID, device_name = 2052, '체육관 B1F 탁구장 배기휀   전기계측기'  # 작동없음
devID, device_name = 2053, '체육관 B1F 전기실 급기휀   전기계측기'  # 작동없음
devID, device_name = 2054, '체육관 B1F 전기실 배기휀   전기계측기'  # 작동없음
devID, device_name = 2120, '식당 급기휀   전기계측기'  # 작동없음
devID, device_name = 2121, '식당 배기휀1   전기계측기'  # 정상 패턴
devID, device_name = 2122, '식당 배기휀2   전기계측기'  # 이상 패턴
devID, device_name = 2123, '식당 배기휀3   전기계측기'  # 정상 패턴

Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 전기온수기 급탕

In [ ]:
# 보류

devID, device_name = 4003, '체육관 전기온수기 급탕'
Util.print_tagCD(df, devID)
tag_dict = {90001: '공급온도', 90002: '환수온도', 90003: '순시유량', 90004: '누적유량_차', 90005: '순시열량', 202202: 'validData'}
tag_dict = {90003: '순시유량'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = Util.calc_tagCD_diff(tags[90004])
#tags[90006] = calc_tagCD_diff(tags[90006])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#df_raw = pd.DataFrame(data=tag_data[0]['colec_val'].values, index=tag_data[0]['colec_dt'], columns=['colec_val'])
#df_interpol = Util.resample_time_index_interpolate_NaN_df(df_raw, '15min', 'linear')
#Util.plot_interpolated_data(df_raw, df_interpol, device_name, createFig=False)

#### 가스 계측기

In [ ]:
# B0008,3001,식당 급탕보일러 가스,가스계측기
# B0008,3002,MAIN GHP 가스,가스계측기
# B0008,3003,강당튜브히터 가스,가스계측기
# B0008,3004,본관 보일러급탕 가스,가스계측기
# B0008,3005,유치원 보일러급탕 가스,가스계측기

devID, device_name = 3001, '누적가스량'
Util.print_tagCD(df, devID)
tag_dict = {70002: '보정적산량_차'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[70002] = Util.calc_tagCD_diff(tags[70002])  # '보정적산량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 에어컨

In [5]:
# EHP-1에 연결된 에어컨 목록록
#devIDs = [6011, 6012, 6047, 6048, 6085, 6086, 6087, 6088, 6089, 6090, 6091, 6105, 6106, 6117, 6054, 6056, 6058, 6060, 6062]
#device_names = ['체육관후면좌_후210', '체육관전면좌_후210', '체육관후면우_후210', '체육관전면우_후210', '세척실_급103', '조리실2_급105', '조리실1_급105', '영양사실_급106', '휴게실_급109-1', '전처리실_급109', '식품창고_급107', '식당중2_급101', '식당중1_급101', '식당후면2_급101', '식당전4_급101', '식당전3_급101', '식당전2_급101', '식당전1_급101', '식당중3_급101']
#devIDs_NA = [6141, 6142, 6143, 6119, 6121]
#device_names = ['체육관교사실_후211', '체육관방송실_후214', '체육관준비실_후213', '식당후면1_급101', '이벤트식당_급101-1']

# EHP-2에 연결된 에어컨 목록
#devIDs = [6033, 6034, 6035, 6036, 6037, 6038, 6039, 6055, 6057, 6059, 6061, 6063, 6064, 6092, 6096, 6098, 6100, 6102]
#device_names = ["행정실2_본203", "행정실1_본203", "탕비실_본204-2", "교무실2_본202", "교무실1_본202", "교장실_본201", "학교운영위원회실_본212", "2-1반_본304", "2-1반_본304", "전산실_본303", "인쇄실_본302", "스튜디오_본301-1", "방송실_본301", "위클래스_후201", "5학년교사연구실_본410", "2학년교사연구실_본311", "위클래스_본211", "1학년교사연구실_본109"]

# EHP-4에 연결된 에어컨 목록
devIDs = [6116, 6002, 6003, 6004, 6005, 6006]
device_names = ['아라반_후103', '유치원복도1', '유치원교사실_후108', '누리반_후107', '한울반_후106', '가람반_후105']
devIDs_NA = [6118, 6120, 6122, 6123, 6144]
device_names_NA = ['마루반_후102', '유치원조리실_후101', '유희실', '유치원도서실', '유치원복도2']

# 관찰 구간
#devID==6001~6083, 6085~6149 "에어컨", tagCD==?????? '??????', anomaly_detection(), vacation_detection(), optimal_control()
#100001	운전선택탭  off: 0, on: 1  <-- 이산치인데 데이터누락이 많아서 정보가치 없음
#100002	냉난방모드	cool: 0, heat: 1, dry: 2, fan: 3, auto: 4, normal: 5, env: 6, sleep: 6
#100003	현재온도	현재온도
#100004	희망온도	냉방 18.0~30.0, 난방 16.0~30.0
#100005	풍량        low: 0, med: 1, mid: 1, high: 2, auto: 3
#100006	리모컨허용  level: 2, true: 1, false: 0
#100007	에러	    indoor: 0, outdoor: 1, caur: 2, trans: 3

#for devID, device_name in zip(devIDs, device_names):
for devID, device_name in zip([6009], ['1-4반_본106']):
    #devID, device_name = 6009, '1-4반_본106'
    print(f'에어컨: {devID=} {device_name=}')
    Util.print_tagCD(df, devID)
    tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도', 100005: '풍량', 100006: '리모컨허용', 100007: '에러', 202202: 'validData'}
    tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도'}
    # dictionary를 이용해 각 태그 데이터를 선택
    tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    # 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
    #tags[90004] = calc_tagCD_diff(tags[90004])

    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [tag_dict[key] for key in tag_dict.keys()]

    Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=True, W=None, H=700)

에어컨: devID=6009 device_name='1-4반_본106'
tagCD list:  [100001 100002 100003 100004 100005 100006 100007 202202]
